In [ ]:
케라스의 콜백을 사용하여 검증 손실이 향상되지 않을 때 훈련을 멈춘다.
콜백은 모델의 fit()메서드가 호출될 때 전달되는 객체(특정 메서드를 구현한 클래스 객체)

훈련하는 동안 모델은 여러 지점에서 콜백을 호출한다. 
콜백은 모델의 상태와 성능에 대한 모든 정보에 접근하고 훈련 중지, 모델 저장, 가중치 적재 또는 모델 상태 변경 등을 처리할 수 있다.

콜백을 사용하는 몇 가지 사례.
1) 모델 체크포인트 저장 : 훈련하는 동안 어떤 지점에서 모델의 현재 가중치 저장
2) 조기 종료(Early Stopping) : 검증 손실이 더 이상 향상되지 않을 때 훈련을 중지한다.(훈련하는 동안 얻은 가장 좋은 모델 저장)
3) 훈련하는 동안 하이퍼파라미터 값을 동적으로 조정 : 옵티마이저의 학습률 같은 경우
4) 훈련과 검증 지표를 로그에 기록하거나 모델이 학습한 표현이 업데이트 될 때마다 시각화

tensorflow.keras.callbacks 문서 참조

In [ ]:
# ModelCheckpoint EarlyStopping Callback

import tensorflow as tf

from tensorflow import keras

callback_list = [tf.keras.callbacks.EarlyStopping(monitor = 'val_acc',
                                                  patience = 1,), # 1에포크 보다 길게(2에포크 동안)정확도가 향상되지 않으면 중지
                tf.keras.callbacks.ModelCheckpoint(
                    filepath = 'my_model.h5',
                    monitor = 'val_loss',
                    save_best_only = True,)] 
# epoch마다 가중치 저장, monitor 기본값은 'val_loss', 정확도가 향상되지 않으면 모델 파일 overwrite하지 않는다.

model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

model.fit(x, y,
          epochs = 10,
          batch_size = 32,
          callbacks = callbacks_list,
          validation_data = (x_val, y_val))

In [ ]:
# ReduceLROnPlateau Callback
callback_list = [tf.keras.callbacks.ReduceLRonPlateau(monitor = 'val_loss', factor = 0.1, patience = 10,)]
    # 검증손실이 10에포크 동안 좋아지지 않으면 콜백이 호출되어 학습률을 10배로 줄인다.

model.fit(x, y, epochs = 10, batch_size = 32, callbacks = callbacks_list, validation = (x_val y_val))




# 자신만의 콜백 만들기

on_epoch_begin = 각 에포크가 시작할 때 호출
on_epoch_end = 각 에포크가 끝날 때 호출

on_batch_begin = 각 배치 처리가 시작되기 전에 호출
on_batch_end = 각 배치 처리가 끝난 후에 호출

on_train_begin = 훈련이 시작될 때 호출
on_train_end = 훈련이 끝날 때 호출

이 메서드들은 모두 logs 매개변수와 함께 호출 - 이 매개변수에는 이전 배치, 에포크에 대한 훈련과 검증 측정값이 담겨 있는 딕셔너리 전달

또한 callback은 다음 속성을 참조할 수 있다.
self.model = 콜백을 호출하는 모델 객체
self.validation_data = fit() 메서드에 전달된 검증 데이터




# 매 에포크의 끝에서 검증 세트의 첫 번째 샘플로 모델에 있는 모든 층의 활성화 출력을 계산하여 (넘파이 배열로) 디스크에 저장하는 자작 콜백의 예시

import tensorflow as tf
import numpy as np

from tensorflow import keras

class ActivationLogger(tf.keras.callbacks.Callback):
    
    def set_model(self, model): # 호출하는 모델에 대한 정보를 전달하기 위해 훈련하기 전에 호출
        self.model = model
        layers_outputs = [layer.output for layer in model.layers]
        self.activations_model = tf.keras.models.Model[(model.input, layer_outputs)] 
        # 각 층의 활성화 출력을 반환하는 Model 객체
        
    def on_epoch_end(self, epoch, logs = None):
        if self.validation_data is None:
            raise RuntimeError('Requires validation_data.')
            
        validation_sample = self.validation_data[0][0:1]
        # 검증 데이터의 첫 번째 샘플을 가져온다, 첫 번째 원소는 입력 데이터, 두 번째 원소는 레이블
        activations = self.activations_model.predict(validation_sample)
        f = open('activations_at_epoch_' + str(epoch) + '.npz', 'wb')
        np.savez(f, activations)
        f.close()